# Import Modules

In [19]:
import warnings
warnings.filterwarnings('ignore')

from src import detect_faces, show_bboxes
from PIL import Image

import torch
from torchvision import transforms, datasets
import numpy as np
import os

# Path Definition

In [20]:
dataset_path = '../Dataset/emotiw/'
face_coordinates_directory = '../Dataset/FaceCoordinates/'
processed_dataset_path = '../Dataset/CroppedFaces/'

# Load Train and Val Dataset

In [21]:
image_datasets = {x : datasets.ImageFolder(os.path.join(dataset_path, x))
                    for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
class_names

['Negative', 'Neutral', 'Positive']

In [25]:
training_dataset = image_datasets['train']
validation_dataset = image_datasets['val']

In [26]:
neg_train = sorted(os.listdir(dataset_path + 'train/Negative/'))
neu_train = sorted(os.listdir(dataset_path + 'train/Neutral/'))
pos_train = sorted(os.listdir(dataset_path + 'train/Positive/'))

neg_val = sorted(os.listdir(dataset_path + 'val/Negative/'))
neu_val = sorted(os.listdir(dataset_path + 'val/Neutral/'))
pos_val = sorted(os.listdir(dataset_path + 'val/Positive/'))

In [27]:
neg_train_filelist = [x.split('.')[0] for x in neg_train]
neu_train_filelist = [x.split('.')[0] for x in neu_train]
pos_train_filelist = [x.split('.')[0] for x in pos_train]

neg_val_filelist = [x.split('.')[0] for x in neg_val]
neu_val_filelist = [x.split('.')[0] for x in neu_val]
pos_val_filelist = [x.split('.')[0] for x in pos_val]

In [28]:
print(neg_train_filelist[:10])
print(neu_train_filelist[:10])
print(pos_train_filelist[:10])

print(neg_val_filelist[:10])
print(neu_val_filelist[:10])
print(pos_val_filelist[:10])

['neg_1', 'neg_10', 'neg_100', 'neg_1000', 'neg_1001', 'neg_1002', 'neg_1003', 'neg_1004', 'neg_1005', 'neg_1006']
['neu_1', 'neu_10', 'neu_100', 'neu_1000', 'neu_1001', 'neu_1002', 'neu_1003', 'neu_1004', 'neu_1005', 'neu_1006']
['pos_1', 'pos_10', 'pos_100', 'pos_1000', 'pos_1001', 'pos_1002', 'pos_1003', 'pos_1004', 'pos_1005', 'pos_1006']
['neg_1', 'neg_10', 'neg_100', 'neg_1000', 'neg_1001', 'neg_1002', 'neg_1003', 'neg_1004', 'neg_1005', 'neg_1006']
['neu_1', 'neu_10', 'neu_100', 'neu_1000', 'neu_1001', 'neu_1002', 'neu_1003', 'neu_1004', 'neu_1005', 'neu_1006']
['pos_1', 'pos_10', 'pos_100', 'pos_1000', 'pos_1001', 'pos_1002', 'pos_1003', 'pos_1004', 'pos_1005', 'pos_1006']


In [29]:
train_filelist = neg_train_filelist + neu_train_filelist + pos_train_filelist
val_filelist = neg_val_filelist + neu_val_filelist + pos_val_filelist

In [30]:
print(len(training_dataset))
print(len(validation_dataset))

9815
4346


# Crop Faces

In [33]:
for i in range(len(training_dataset)):
    try:
        image, label = training_dataset[i]
        face_list = []
        landmarks_new_coordinates = []
        if label == 0:
            if os.path.isfile(processed_dataset_path + 'train/Negative/' + train_filelist[i] + '.npz'):
                print(train_filelist[i] + ' Already present')
                continue

            bbox_lm = np.load(face_coordinates_directory + 'train/Negative/' + train_filelist[i] +'.npz')
            bounding_boxes = bbox_lm['a']
            if bounding_boxes.size == 0 or (bounding_boxes[0] == 0).all():
                print("No bounding boxes for " + train_filelist[i] + ". Adding empty file for the same")
                np.savez(processed_dataset_path + 'train/Negative/' + train_filelist[i], a = np.zeros(1), b = np.zeros(1))
                continue
            landmarks = bbox_lm['b']

            for j in range(len(bounding_boxes)):
                bbox_coordinates = bounding_boxes[j]
                landmark = landmarks[j]
                img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))

                x = bbox_coordinates[0]
                y = bbox_coordinates[1]
                
                for k in range(5):
                    landmark[k] -= x
                    landmark[k+5] -= y
                img_face = np.array(img_face)
                landmark = np.array(landmark)


                if len(face_list) != 0:
                    if img_face.shape[0] == face_list[-1].shape[0]:
                        img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
                        img_face = np.array(img_face)
                        landmark +=1

                face_list.append(img_face)
                landmarks_new_coordinates.append(landmark)
            face_list = np.asarray(face_list)
            landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
            np.savez(processed_dataset_path + 'train/Negative/' + train_filelist[i], a = face_list, b = landmarks_new_coordinates)

        elif label == 1:

            if os.path.isfile(processed_dataset_path + 'train/Neutral/' + train_filelist[i] + '.npz'):
                print(train_filelist[i] + ' Already present')
                continue

            bbox_lm = np.load(face_coordinates_directory + 'train/Neutral/' + train_filelist[i] +'.npz')
            bounding_boxes = bbox_lm['a']
            if bounding_boxes.size == 0 or (bounding_boxes[0] == 0).all():
                print("No bounding boxes for " + train_filelist[i] + ". Adding empty file for the same")
                np.savez(processed_dataset_path + 'train/Neutral/' + train_filelist[i], a = np.zeros(1), b = np.zeros(1))
                continue
            landmarks = bbox_lm['b']

            for j in range(len(bounding_boxes)):
                bbox_coordinates = bounding_boxes[j]
                landmark = landmarks[j]
                img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
   
                x = bbox_coordinates[0]
                y = bbox_coordinates[1]
        
                for k in range(5):
                    landmark[k] -= x
                    landmark[k+5] -= y
                img_face = np.array(img_face)
                landmark = np.array(landmark)

                if len(face_list) != 0:
                    if img_face.shape[0] == face_list[-1].shape[0]:
                        img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
                        img_face = np.array(img_face)
                        landmark += 1

                face_list.append(img_face)
                landmarks_new_coordinates.append(landmark)
            face_list = np.asarray(face_list)
            landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
            np.savez(processed_dataset_path + 'train/Neutral/' + train_filelist[i], a = face_list, b = landmarks_new_coordinates)

        else:
            if os.path.isfile(processed_dataset_path + 'train/Positive/' + train_filelist[i] + '.npz'):
                print(train_filelist[i] + ' Already present')
                continue

            bbox_lm = np.load(face_coordinates_directory + 'train/Positive/' + train_filelist[i] +'.npz')
            bounding_boxes = bbox_lm['a']
            if bounding_boxes.size == 0 or (bounding_boxes[0] == 0).all():
                print("No bounding boxes for " + train_filelist[i] + ". Adding empty file for the same")
                np.savez(processed_dataset_path + 'train/Positive/' + train_filelist[i], a = np.zeros(1), b = np.zeros(1))
                continue
            landmarks = bbox_lm['b']

            for j in range(len(bounding_boxes)):
                bbox_coordinates = bounding_boxes[j]
                landmark = landmarks[j]
                img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
    
                x = bbox_coordinates[0]
                y = bbox_coordinates[1]
            
                for k in range(5):
                    landmark[k] -= x
                    landmark[k+5] -= y
                img_face = np.array(img_face)
                landmark = np.array(landmark)

                if len(face_list) != 0:
                    if img_face.shape[0] == face_list[-1].shape[0]:
                        img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
                        img_face = np.array(img_face)
                        landmark += 1

                face_list.append(img_face)
                landmarks_new_coordinates.append(landmark)
            face_list = np.asarray(face_list)
            landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
            np.savez(processed_dataset_path + 'train/Positive/' + train_filelist[i], a = face_list, b = landmarks_new_coordinates)

        if i % 100 == 0:
            print(i)
    except:
        print("Error/interrput at validation dataset file " + val_filelist[i])
        print(bounding_boxes)
        print(landmarks)
        print(bounding_boxes.shape)
        print(landmarks.shape)
        break

neg_1 Already present
neg_10 Already present
neg_100 Already present
neg_1000 Already present
neg_1001 Already present
neg_1002 Already present
neg_1003 Already present
neg_1004 Already present
neg_1005 Already present
neg_1006 Already present
neg_1007 Already present
neg_1008 Already present
neg_1009 Already present
neg_101 Already present
neg_1010 Already present
neg_1011 Already present
neg_1012 Already present
neg_1013 Already present
neg_1014 Already present
neg_1015 Already present
neg_1016 Already present
neg_1017 Already present
neg_1018 Already present
neg_1019 Already present
neg_102 Already present
neg_1020 Already present
neg_1021 Already present
neg_1022 Already present
neg_1023 Already present
neg_1024 Already present
neg_1025 Already present
neg_1026 Already present
neg_1027 Already present
neg_1028 Already present
neg_1029 Already present
neg_103 Already present
neg_1030 Already present
neg_1031 Already present
neg_1032 Already present
neg_1033 Already present
neg_1034 

neg_1298 Already present
neg_1299 Already present
neg_13 Already present
neg_130 Already present
neg_1300 Already present
neg_1301 Already present
neg_1302 Already present
neg_1303 Already present
neg_1304 Already present
neg_1305 Already present
neg_1306 Already present
neg_1307 Already present
neg_1308 Already present
neg_1309 Already present
neg_131 Already present
neg_1310 Already present
neg_1311 Already present
neg_1312 Already present
neg_1313 Already present
neg_1314 Already present
neg_1315 Already present
neg_1316 Already present
neg_1317 Already present
neg_1318 Already present
neg_1319 Already present
neg_132 Already present
neg_1320 Already present
neg_1321 Already present
neg_1322 Already present
neg_1323 Already present
neg_1324 Already present
neg_1325 Already present
neg_1326 Already present
neg_1327 Already present
neg_1328 Already present
neg_1329 Already present
neg_133 Already present
neg_1330 Already present
neg_1331 Already present
neg_1332 Already present
neg_13

neg_1601 Already present
neg_1602 Already present
neg_1603 Already present
neg_1604 Already present
neg_1605 Already present
neg_1606 Already present
neg_1607 Already present
neg_1608 Already present
neg_1609 Already present
neg_161 Already present
neg_1610 Already present
neg_1611 Already present
neg_1612 Already present
neg_1613 Already present
neg_1614 Already present
neg_1615 Already present
neg_1616 Already present
neg_1617 Already present
neg_1618 Already present
neg_1619 Already present
neg_162 Already present
neg_1620 Already present
neg_1621 Already present
neg_1622 Already present
neg_1623 Already present
neg_1624 Already present
neg_1625 Already present
neg_1626 Already present
neg_1627 Already present
neg_1628 Already present
neg_1629 Already present
neg_163 Already present
neg_1630 Already present
neg_1631 Already present
neg_1632 Already present
neg_1633 Already present
neg_1634 Already present
neg_1635 Already present
neg_1636 Already present
neg_1637 Already present
neg

neg_1908 Already present
neg_1909 Already present
neg_191 Already present
neg_1910 Already present
neg_1911 Already present
neg_1912 Already present
neg_1913 Already present
neg_1914 Already present
neg_1915 Already present
neg_1916 Already present
neg_1917 Already present
neg_1918 Already present
neg_1919 Already present
neg_192 Already present
neg_1920 Already present
neg_1921 Already present
neg_1922 Already present
neg_1923 Already present
neg_1924 Already present
neg_1925 Already present
neg_1926 Already present
neg_1927 Already present
neg_1928 Already present
neg_1929 Already present
neg_193 Already present
neg_1930 Already present
neg_1931 Already present
neg_1932 Already present
neg_1933 Already present
neg_1934 Already present
neg_1935 Already present
neg_1936 Already present
neg_1937 Already present
neg_1938 Already present
neg_1939 Already present
neg_194 Already present
neg_1940 Already present
neg_1941 Already present
neg_1942 Already present
neg_1943 Already present
neg_

neg_2210 Already present
neg_2211 Already present
neg_2212 Already present
neg_2213 Already present
neg_2214 Already present
neg_2215 Already present
neg_2216 Already present
neg_2217 Already present
neg_2218 Already present
neg_2219 Already present
neg_222 Already present
neg_2220 Already present
neg_2221 Already present
neg_2222 Already present
neg_2223 Already present
neg_2224 Already present
neg_2225 Already present
neg_2226 Already present
neg_2227 Already present
neg_2228 Already present
neg_2229 Already present
neg_223 Already present
neg_2230 Already present
neg_2231 Already present
neg_2232 Already present
neg_2233 Already present
neg_2234 Already present
neg_2235 Already present
neg_2236 Already present
neg_2237 Already present
neg_2238 Already present
neg_2239 Already present
neg_224 Already present
neg_2240 Already present
neg_2241 Already present
neg_2242 Already present
neg_2243 Already present
neg_2244 Already present
neg_2245 Already present
neg_2246 Already present
neg

...

pos_3206 Already present
pos_3207 Already present
pos_3208 Already present
pos_3209 Already present
pos_321 Already present
pos_3210 Already present
pos_3211 Already present
pos_3212 Already present
pos_3213 Already present
pos_3214 Already present
pos_3215 Already present
pos_3216 Already present
pos_3217 Already present
pos_3218 Already present
pos_3219 Already present
pos_322 Already present
pos_3220 Already present
pos_3221 Already present
pos_3222 Already present
pos_3223 Already present
pos_3224 Already present
pos_3225 Already present
pos_3226 Already present
pos_3227 Already present
pos_3228 Already present
pos_3229 Already present
pos_323 Already present
pos_3230 Already present
pos_3231 Already present
pos_3232 Already present
pos_3233 Already present
pos_3234 Already present
pos_3235 Already present
pos_3236 Already present
pos_3237 Already present
pos_3238 Already present
pos_3239 Already present
pos_324 Already present
pos_3240 Already present
pos_3241 Already present
pos_

pos_3502 Already present
pos_3503 Already present
pos_3504 Already present
pos_3505 Already present
pos_3506 Already present
pos_3507 Already present
pos_3508 Already present
pos_3509 Already present
pos_351 Already present
pos_3510 Already present
pos_3511 Already present
pos_3512 Already present
pos_3513 Already present
pos_3514 Already present
pos_3515 Already present
pos_3516 Already present
pos_3517 Already present
pos_3518 Already present
pos_3519 Already present
pos_352 Already present
pos_3520 Already present
pos_3521 Already present
pos_3522 Already present
pos_3523 Already present
pos_3524 Already present
pos_3525 Already present
pos_3526 Already present
pos_3527 Already present
pos_3528 Already present
pos_3529 Already present
pos_353 Already present
pos_3530 Already present
pos_3531 Already present
pos_3532 Already present
pos_3533 Already present
pos_3534 Already present
pos_3535 Already present
pos_3536 Already present
pos_3537 Already present
pos_3538 Already present
pos

pos_3800 Already present
pos_3801 Already present
pos_3802 Already present
pos_3803 Already present
pos_3804 Already present
pos_3805 Already present
pos_3806 Already present
pos_3807 Already present
pos_3808 Already present
pos_3809 Already present
pos_381 Already present
pos_3810 Already present
pos_3811 Already present
pos_3812 Already present
pos_3813 Already present
pos_3814 Already present
pos_3815 Already present
pos_3816 Already present
pos_3817 Already present
pos_3818 Already present
pos_3819 Already present
pos_382 Already present
pos_3820 Already present
pos_3821 Already present
pos_3822 Already present
pos_3823 Already present
pos_3824 Already present
pos_3825 Already present
pos_3826 Already present
pos_3827 Already present
pos_3828 Already present
pos_3829 Already present
pos_383 Already present
pos_3830 Already present
pos_3831 Already present
pos_3832 Already present
pos_3833 Already present
pos_3834 Already present
pos_3835 Already present
pos_3836 Already present
pos

pos_529 Already present
pos_53 Already present
pos_530 Already present
pos_531 Already present
pos_532 Already present
pos_533 Already present
pos_534 Already present
pos_535 Already present
pos_536 Already present
pos_537 Already present
pos_538 Already present
pos_539 Already present
pos_54 Already present
pos_540 Already present
pos_541 Already present
pos_542 Already present
pos_543 Already present
pos_544 Already present
pos_545 Already present
pos_546 Already present
pos_547 Already present
pos_548 Already present
pos_549 Already present
pos_55 Already present
pos_550 Already present
pos_551 Already present
pos_552 Already present
pos_553 Already present
pos_554 Already present
pos_555 Already present
pos_556 Already present
pos_557 Already present
pos_558 Already present
pos_559 Already present
pos_56 Already present
pos_560 Already present
pos_561 Already present
pos_562 Already present
pos_563 Already present
pos_564 Already present
pos_565 Already present
pos_566 Already pres

pos_847 Already present
pos_848 Already present
pos_849 Already present
pos_85 Already present
pos_850 Already present
pos_851 Already present
pos_852 Already present
pos_853 Already present
pos_854 Already present
pos_855 Already present
pos_856 Already present
pos_857 Already present
pos_858 Already present
pos_859 Already present
pos_86 Already present
pos_860 Already present
pos_861 Already present
pos_862 Already present
pos_863 Already present
pos_864 Already present
pos_865 Already present
pos_866 Already present
pos_867 Already present
pos_868 Already present
pos_869 Already present
pos_87 Already present
pos_870 Already present
pos_871 Already present
pos_872 Already present
pos_873 Already present
pos_874 Already present
pos_875 Already present
pos_876 Already present
pos_877 Already present
pos_878 Already present
pos_879 Already present
pos_88 Already present
pos_880 Already present
pos_881 Already present
pos_882 Already present
pos_883 Already present
pos_884 Already pres